# Neural information retrieval

In [5]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt             
import seaborn as sns
import torch
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim

## EDA

In [2]:
chairs_df = pd.read_csv('../data/kancelarska-kresla.heureka.cz.csv')

In [3]:
chairs_df.sample(5)

,name,categories,price,rating,specification
320,Alba Kent Exclusive,"['Bydlení a doplňky', 'Nábytek', 'Židle a stol...",8763,100.0,Elegantní křeslo do kanceláře v základním prov...
50,Antares Scope,"['Bydlení a doplňky', 'Nábytek', 'Židle a stol...",5990,100.0,\n\tDesignová židle s opěrkou hlavy a područka...
774,Corping Human,"['Bydlení a doplňky', 'Nábytek', 'Židle a stol...",4299,90.0,
263,SEDIA Q031,"['Bydlení a doplňky', 'Nábytek', 'Židle a stol...",3890,94.0,Křeslo Q031
32,Alba Fuxo S-Line,"['Bydlení a doplňky', 'Nábytek', 'Židle a stol...",3796,97.0,Rostoucí židle Fuxo


In [4]:
chairs_df.isnull().sum()

name              0
categories        0
price             0
rating           13
specification     6
dtype: int64

In [5]:
# replace missing values of rating with mean
chairs_df['rating'] = chairs_df['rating'].fillna(chairs_df['rating'].mean())

In [6]:
# replace missing specification with empty string
chairs_df['specification'] = chairs_df['specification'].fillna('')

In [15]:
chairs_df['name'] = chairs_df['name'].astype(str)
chairs_df['specification'] = chairs_df['specification'].astype(str)

## Model

In [9]:
backbone = 'mixedbread-ai/mxbai-embed-large-v1'
model = SentenceTransformer(backbone)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/112k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/677 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [23]:
# TEST
query = 'křeslo které stojí více než 6000 Kč'
query_embedding = model.encode([query])
docs = []
for i in range(10):
    tmp = f'Jméno: {chairs_df["name"].iloc[i]}, Cena: {chairs_df["price"].iloc[i]} Kč, Popis: {chairs_df["specification"].iloc[i]}'
    docs.append(tmp)
print(f'Docs: {docs}')
doc_embeddings = model.encode(docs)

similarities = cos_sim(query_embedding[0], doc_embeddings)
print(f'Similarities: {similarities}')

Docs: ['Jméno: Superkancl Optima, Cena: 2995 Kč, Popis: Kancelářská židle OPTIMA PDH s područkami a podhlavníkem má vše pro pohodlné sezení. Její předností je široký sedák, ten je čalouněný kvalitní prodyšnou látkou. Výplň sedáku je z pěny s vysokou hustotou a odolností proti prosezení. Ani při delším sezení se nekroutí a umožňuje tak optimální komfort sezení. Opěradlo zad s prodyšným potahem ze síťoviny podpírá páteř a díky svému ergonomickému tvaru předchází bolestem zad. Je doplněné o bederní opěrku a zakončené pevným podhlavníkem. Ruce si můžete pohodlně položit na čalouněné područky, které se dají odklopit směrem nahoru. Kancelářskou židli tak snadno zasunete pod stůl. Svojí velikostí je vhodná pro osoby s výškou do 185 cm. Židle má kvalitní houpací mechanismus s nastavením síly houpání, ta se reguluje velkým plastovým šroubem umístěným pod sedákem. Výška sezení se upravuje plynule páčkou. Je použitý kvalitní plynový píst s certifikátem BIFMA Class 3, chromovaný kříž má pogumova

In [13]:
len(query_embedding[0])

1024